# Permutation Entropy

Define simple complexity measures which are easily calculated for any time of time series where regular, chaotic, noisy or reality based.

## Example

Given a series with 7 values:
$$x = (4,7,9,10,6,11,3)$$

Organize 6 pairs of neighbors, according to their relative values, and find 4 pairs/6 with $x_t < x_{t+1}$, represented
by permutation $01$ and 2 pairs/6 with $x_t > x_{t+1}$ represented by $10$. Define permutation entropy of order $n = 2$
as a measure of probabilities of permutations 01 and 10. $$H(2) = -(4/6)log(4/6) - (2/6)log(2/6) \approx 0.918$$

Since log is usually with base 2, so $H$ is given in bit.

Next, compare three consecutive values, (4,7,9), (7,9,10) and so on. (4,7,9) and (7,9,10) represent permutation 012 due to increasing order.
$$H(3) = -2(2/5)log(2/5) - (1/5)log(1/5) \approx 1.522$$

## Formula

Consider a time series $\{x_t\}_{t=1,...,T}$. We study all $n!$ permutations $\pi$ of order $n$ which are considered as possible order types of n different numbers. For each $\pi$ we determine relative frequence (# for number)

$$p(\pi) = \frac{\#\{t|t \leq T-n, (x_{t+1},...,x_{t+n}) \text{ has type } \pi\}}{T-n+1}$$

This estimates the frequency of $\pi$ as good as possible for a finite series of values.

The permutation entropy of order $n \geq 2$ is:
$$H(n) = - \sum p(\pi) log p(\pi)$$
where sum runs over all $n!$ permutations $\pi$ of order $n$.

- Define series, order and delay

In [1]:
import numpy as np
import pandas as pd

series = np.array([4, 7, 9, 10, 6, 11, 3])
order = 3
delay = 1
normalized = False

def _embed(x, order=3, delay=1):
    N = len(x)
    Y = np.zeros((order, N - (order - 1) * delay))
    for i in range(order):
        Y[i] = x[i * delay:i * delay + Y.shape[1]]
    return Y.T

- Prepare hash multiplier to later compute unique hash value for each for each permutation $\pi$

In [2]:
ran_order = range(order) # 0, 1, 2
hashmult = np.power(order, ran_order)
hashmult

array([1, 3, 9], dtype=int32)

- Embed the series with order = 3, similar to other entropy measures

In [3]:
embed = _embed(series, order=order, delay=delay)
embed

array([[ 4.,  7.,  9.],
       [ 7.,  9., 10.],
       [ 9., 10.,  6.],
       [10.,  6., 11.],
       [ 6., 11.,  3.]])

- Convert each permutation to index representation
- Sort entropy index
`[9, 10, 6]` when sorted by index will be `[2, 0, 1]`

In [4]:
sorted_idx = embed.argsort(kind='quicksort')
sorted_idx

array([[0, 1, 2],
       [0, 1, 2],
       [2, 0, 1],
       [1, 0, 2],
       [2, 0, 1]], dtype=int64)

- Associate unique integer to each permutation
`[0, 1, 2] * [1, 3, 9] = [0, 3, 18]` summed across axis 1 to be `21`

In [5]:
hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
hashval

array([21, 21, 11, 19, 11], dtype=int64)

- Count unique permutations

In [6]:
_, c = np.unique(hashval, return_counts=True)
c

array([2, 1, 2], dtype=int64)

- Calculate probability for each permutation

In [7]:
p = np.true_divide(c, c.sum())
p

array([0.4, 0.2, 0.4])

- Calculate Permutation Entropy based on formula:

$$H(n) = - \sum p(\pi) log p(\pi)$$

In [8]:
pe = -np.multiply(p, np.log2(p)).sum()
print("Perm Ent: ", pe)

Perm Ent:  1.5219280948873621


- If we want to obtain normalized entropy:
$$\frac{H(n)}{logn!}$$

In [9]:
import math

normalized_pe = pe / np.log2(math.factorial(order))
print("Normalized Perm Ent: ", normalized_pe)


Normalized Perm Ent:  0.5887621559162939
